In [1]:
import mysql.connector
from datetime import datetime
import tkinter as tk
from tkinter import ttk 
from tkinter import messagebox, simpledialog


def conectar():
    """Establece la conexión con la base de datos MySQL."""
    try:
        conn = mysql.connector.connect(
            host="shortline.proxy.rlwy.net",
            port=47316,
            user="root",
            password="cHHzHOJVpxFjPXLJsUQPHMNiZhCECXfJ", 
            database="gamestore_db"
        )
        return conn
    except mysql.connector.Error as err:
        print(f"Error de conexión a la base de datos (desde conectar()): {err}")
        return None

In [2]:
class GameStoreManager:
    def __init__(self):
        self.db_conn = conectar()
        if self.db_conn:
            print("GameStoreManager: Conectado a la base de datos.") 
            self._crear_tablas_si_no_existen()
        else:
            print("GameStoreManager: No se pudo conectar a la base de datos.")
            
    def _execute_query(self, query, params=None, fetch=None, commit=False, is_ddl=False):
        if not self.db_conn or not self.db_conn.is_connected():
            print("GameStoreManager: Intentando reconectar...")
            self.db_conn = conectar()
            if not self.db_conn:
                print("GameStoreManager: Fallo al reconectar.")
                return None if fetch else False
        
        use_dictionary_cursor = isinstance(fetch, str) and 'dict' in fetch
        cursor = None
        try:
            cursor = self.db_conn.cursor(dictionary=use_dictionary_cursor)
            cursor.execute(query, params or ())
            if commit or is_ddl:
                self.db_conn.commit()
            if fetch:
                if fetch == 'one' or fetch == 'dict_one':
                    return cursor.fetchone()
                elif fetch == 'all' or fetch == 'dict_all':
                    return cursor.fetchall()
            return True
        except mysql.connector.Error as err:
            print(f"GameStoreManager DB Error: {err}") 
            if self.db_conn and commit: # Only rollback if commit was intended
                try: self.db_conn.rollback()
                except mysql.connector.Error as roll_err: print(f"Rollback Error: {roll_err}")
            return None if fetch else False
        finally:
            if cursor: cursor.close()

    def _crear_tablas_si_no_existen(self):
        queries = {
            "videojuegos": """
                CREATE TABLE IF NOT EXISTS videojuegos (
                    id INT AUTO_INCREMENT PRIMARY KEY, nombre VARCHAR(255) NOT NULL,
                    categoria VARCHAR(100), precio DECIMAL(10, 2) NOT NULL,
                    consola VARCHAR(100), genero VARCHAR(100)
                );""",
            "clientes": """
                CREATE TABLE IF NOT EXISTS clientes (
                    id INT AUTO_INCREMENT PRIMARY KEY, nombre VARCHAR(255) NOT NULL,
                    email VARCHAR(255) UNIQUE
                );""",
            "ventas": """
                CREATE TABLE IF NOT EXISTS ventas (
                    id INT AUTO_INCREMENT PRIMARY KEY, 
                    id_cliente INT,
                    nombre_videojuego_vendido VARCHAR(255) NOT NULL,
                    precio_venta DECIMAL(10, 2) NOT NULL,
                    fecha_venta TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    FOREIGN KEY (id_cliente) REFERENCES clientes(id) ON DELETE SET NULL
                );"""
        }
        for tabla, query in queries.items():
            if self._execute_query(query, is_ddl=True):
                print(f"GameStoreManager: Tabla '{tabla}' verificada/creada.")
            else:
                print(f"GameStoreManager: Error al crear/verificar tabla '{tabla}'.")
    
    def is_connected(self):
        return self.db_conn and self.db_conn.is_connected()

    def add_videojuego_gui(self, nombre, categoria, precio, consola, genero):
        if not nombre or precio is None:
            return False, "Nombre y Precio son obligatorios."
        try:
            precio_float = float(precio)
            if precio_float < 0:
                return False, "El precio no puede ser negativo."
        except ValueError:
            return False, "El precio debe ser un número válido."

        query = "INSERT INTO videojuegos (nombre, categoria, precio, consola, genero) VALUES (%s, %s, %s, %s, %s)"
        params = (nombre, categoria, precio_float, consola, genero)
        if self._execute_query(query, params, commit=True):
            return True, f"Videojuego '{nombre}' agregado exitosamente."
        else:
            return False, f"Error al agregar videojuego '{nombre}'."

    def add_cliente_gui(self, nombre, email):
        if not nombre:
            return False, "El nombre del cliente es obligatorio."
        email_param = email if email else None # Allow empty email to be stored as NULL
        query = "INSERT INTO clientes (nombre, email) VALUES (%s, %s)"
        params = (nombre, email_param)
        if self._execute_query(query, params, commit=True):
            return True, f"Cliente '{nombre}' agregado exitosamente."
        else:
            # More specific error for unique constraint might be helpful if possible,
            # but general error is fine for now.
            return False, f"Error al agregar cliente '{nombre}'. Verifique si el email ya existe."

    def get_catalogo_videojuegos(self):
        query = "SELECT id, nombre, categoria, precio, consola, genero FROM videojuegos ORDER BY nombre"
        return self._execute_query(query, fetch='dict_all')

    def get_clientes_for_selection(self):
        query = "SELECT id, nombre, email FROM clientes ORDER BY nombre"
        return self._execute_query(query, fetch='dict_all')
        
    def get_videojuego_by_id(self, juego_id):
        query = "SELECT id, nombre, precio FROM videojuegos WHERE id = %s"
        return self._execute_query(query, (juego_id,), fetch='dict_one')

    def realizar_venta_gui(self, id_videojuego, nombre_videojuego_vendido, precio_venta, id_cliente):
        # Ensure id_cliente is None if it's not a valid ID (e.g. 0 or some placeholder)
        # Depending on how 'cliente_map' is structured, id_cliente might need validation
        # For now, assume id_cliente is either a valid client ID or None for anonymous/guest.
        # The DB schema allows id_cliente to be NULL.       
        
        cliente_id_param = id_cliente if id_cliente is not None else None # Explicitly handle if None is passed

        query_venta = "INSERT INTO ventas (id_cliente, nombre_videojuego_vendido, precio_venta, fecha_venta) VALUES (%s, %s, %s, %s)"
        fecha_actual = datetime.now()
        params_venta = (cliente_id_param, nombre_videojuego_vendido, precio_venta, fecha_actual)
        
        if not self._execute_query(query_venta, params_venta, commit=True):
            return False, "Error al registrar la venta."

        # Only attempt to remove game if sale was successful
        query_remover_juego = "DELETE FROM videojuegos WHERE id = %s"
        if self._execute_query(query_remover_juego, (id_videojuego,), commit=True):
            return True, f"Venta de '{nombre_videojuego_vendido}' completada y juego removido del catálogo."
        else:
            # This part is problematic: if sale is registered but game removal fails, DB is inconsistent.
            # Ideally, this should be a transaction. For simplicity now, we just report.
            return False, f"Venta registrada, pero ¡ERROR al remover el videojuego '{nombre_videojuego_vendido}'!"

    def get_all_sales_report(self):
        query = """
            SELECT v.id, c.nombre AS nombre_cliente, v.nombre_videojuego_vendido, v.precio_venta, v.fecha_venta
            FROM ventas v
            LEFT JOIN clientes c ON v.id_cliente = c.id
            ORDER BY v.fecha_venta DESC;
        """
        return self._execute_query(query, fetch='dict_all')

    def get_total_revenue_report(self):
        query = "SELECT SUM(precio_venta) AS total FROM ventas"
        resultado = self._execute_query(query, fetch='dict_one')
        if resultado and resultado['total'] is not None:
            return resultado['total'], "Ingresos totales calculados."
        elif resultado and resultado['total'] is None: # No sales yet
            return 0.0, "No hay ventas registradas."
        else: # Error in query
            return None, "Error al calcular ingresos totales."

    def get_sales_by_client_report(self):
        # This query will only show clients who have made purchases.
        # Clients without purchases won't appear.
        query = """
            SELECT c.nombre AS nombre_cliente, COUNT(v.id) AS numero_ventas, SUM(v.precio_venta) AS total_gastado
            FROM clientes c
            JOIN ventas v ON c.id = v.id_cliente 
            GROUP BY c.id, c.nombre
            ORDER BY total_gastado DESC;
        """
        report_data = self._execute_query(query, fetch='dict_all')
        if report_data is None: # Query error
            return None, "Error al generar el informe de ventas por cliente."
        if not report_data: # No sales or no clients with sales
            return [], "No hay ventas registradas o clientes con ventas."
            
        return report_data, "Informe de ventas por cliente generado."


    def cerrar_conexion(self):
        if self.db_conn and self.db_conn.is_connected():
            self.db_conn.close()
            print("GameStoreManager: Conexión cerrada.")


In [ ]:
class GameStoreApp:
    def __init__(self, root_window):
        self.root = root_window
        self.root.title("GameStore Manager")
        self.root.geometry("800x600") 

        self.manager = GameStoreManager()

        if not self.manager.is_connected():
            messagebox.showerror("Error de Conexión", 
                                 "No se pudo conectar a la base de datos.\n"
                                 "La aplicación no funcionará correctamente.\n"
                                 "Revise la consola para más detalles.")
            # Optionally, disable UI elements or close app if connection is critical
        
        
        style = ttk.Style()
        style.theme_use('clam') # Or 'alt', 'default', 'classic'

        # --- Menu Frame ---
        menu_frame = ttk.Frame(self.root, padding="10")
        menu_frame.pack(side=tk.TOP, fill=tk.X)

        ttk.Button(menu_frame, text="Agregar Videojuego", command=self.open_add_videojuego_window).pack(side=tk.LEFT, padx=5)
        ttk.Button(menu_frame, text="Agregar Cliente", command=self.open_add_cliente_window).pack(side=tk.LEFT, padx=5)
        # Removed "Mostrar Catálogo" button as per request
        ttk.Button(menu_frame, text="Realizar Venta", command=self.open_realizar_venta_window).pack(side=tk.LEFT, padx=5)
        ttk.Button(menu_frame, text="Informes de Ventas", command=self.open_informes_window).pack(side=tk.LEFT, padx=5)

        # --- Content Frame ---
        self.content_frame = ttk.Frame(self.root, padding="10")
        self.content_frame.pack(expand=True, fill=tk.BOTH)
        
        # Show catalog by default
        self.show_catalogo_in_content_frame()


    def clear_content_frame(self):
        for widget in self.content_frame.winfo_children():
            widget.destroy()

    def open_add_videojuego_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return

        top = tk.Toplevel(self.root)
        top.title("Agregar Videojuego")
        top.geometry("400x300")

        fields = ["Nombre", "Categoría", "Precio", "Consola", "Género"]
        entries = {}

        form_frame = ttk.Frame(top, padding="10")
        form_frame.pack(expand=True, fill=tk.BOTH)

        for i, field in enumerate(fields):
            ttk.Label(form_frame, text=field + ":").grid(row=i, column=0, padx=5, pady=5, sticky=tk.W)
            entry = ttk.Entry(form_frame, width=30)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky=tk.EW)
            entries[field] = entry
        
        form_frame.grid_columnconfigure(1, weight=1) # Make entry column expandable

        def submit_videojuego():
            data = {field: entries[field].get().strip() for field in fields}
            if not data["Nombre"] or not data["Precio"]:
                messagebox.showerror("Error de Validación", "Nombre y Precio son obligatorios.", parent=top)
                return
            try:
                # Validate price is a number, manager will do more specific validation (e.g. positive)
                float(data["Precio"])
            except ValueError:
                messagebox.showerror("Error de Validación", "El precio debe ser un número.", parent=top)
                return

            success, message = self.manager.add_videojuego_gui(
                data["Nombre"], data["Categoría"], data["Precio"], data["Consola"], data["Género"]
            )
            if success:
                messagebox.showinfo("Éxito", message, parent=top)
                top.destroy()
                self.show_catalogo_in_content_frame() # Refresh catalog view
            else:
                messagebox.showerror("Error", message, parent=top)

        ttk.Button(form_frame, text="Agregar", command=submit_videojuego).grid(row=len(fields), column=0, columnspan=2, pady=10)
        top.transient(self.root) # Keep window on top of main
        top.grab_set() # Modal behavior
        self.root.wait_window(top) # Wait until this window is closed


    def open_add_cliente_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return
            
        top = tk.Toplevel(self.root)
        top.title("Agregar Cliente")
        top.geometry("350x200")

        fields = ["Nombre", "Email"]
        entries = {}
        form_frame = ttk.Frame(top, padding="10")
        form_frame.pack(expand=True, fill=tk.BOTH)

        for i, field in enumerate(fields):
            ttk.Label(form_frame, text=field + ":").grid(row=i, column=0, padx=5, pady=5, sticky=tk.W)
            entry = ttk.Entry(form_frame, width=30)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky=tk.EW)
            entries[field] = entry
        
        form_frame.grid_columnconfigure(1, weight=1)

        def submit_cliente():
            data = {field: entries[field].get().strip() for field in fields}
            if not data["Nombre"]:
                messagebox.showerror("Error de Validación", "El nombre es obligatorio.", parent=top)
                return

            success, message = self.manager.add_cliente_gui(data["Nombre"], data["Email"])
            if success:
                messagebox.showinfo("Éxito", message, parent=top)
                top.destroy()
            else:
                messagebox.showerror("Error", message, parent=top)
        
        ttk.Button(form_frame, text="Agregar", command=submit_cliente).grid(row=len(fields), column=0, columnspan=2, pady=10)
        top.transient(self.root)
        top.grab_set()
        self.root.wait_window(top)

    def show_catalogo_in_content_frame(self, event=None): # event param for potential binding
        self.clear_content_frame()
        if not self.manager.is_connected():
            ttk.Label(self.content_frame, text="No hay conexión a la base de datos para mostrar el catálogo.", foreground="red").pack(pady=20)
            return

        catalogo = self.manager.get_catalogo_videojuegos()

        if catalogo is None: # Indicates a DB error during fetch
            ttk.Label(self.content_frame, text="Error al cargar el catálogo.", foreground="red").pack(pady=20)
            return
        if not catalogo: # Empty list, no games
            ttk.Label(self.content_frame, text="El catálogo de videojuegos está vacío.").pack(pady=20)
            return

        cols = ('ID', 'Nombre', 'Categoría', 'Precio', 'Consola', 'Género')
        tree = ttk.Treeview(self.content_frame, columns=cols, show='headings', selectmode="browse")

        for col in cols:
            tree.heading(col, text=col)
            tree.column(col, width=100, anchor=tk.W) # Default width and anchor
        tree.column('Nombre', width=200)
        tree.column('Precio', anchor=tk.E, width=80) # Align price to the right


        for juego in catalogo:
            tree.insert("", tk.END, values=(
                juego['id'], juego['nombre'], juego.get('categoria', ''), f"${juego['precio']:.2f}", 
                juego.get('consola', ''), juego.get('genero', '')
            ))
        
        # Scrollbars
        vsb = ttk.Scrollbar(self.content_frame, orient="vertical", command=tree.yview)
        hsb = ttk.Scrollbar(self.content_frame, orient="horizontal", command=tree.xview)
        tree.configure(yscrollcommand=vsb.set, xscrollcommand=hsb.set)

        tree.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)
        vsb.pack(side=tk.RIGHT, fill=tk.Y)
        hsb.pack(side=tk.BOTTOM, fill=tk.X)

    # Removed open_show_catalogo_window method as it's no longer used by a button
    # and show_catalogo_in_content_frame is called directly.

    def open_realizar_venta_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return

        top = tk.Toplevel(self.root)
        top.title("Realizar Venta")
        top.geometry("600x450") # Adjusted size for better layout

        # Frame for selections (Game, Client)
        selection_frame = ttk.Frame(top, padding="10")
        selection_frame.pack(fill=tk.X)
        
        # Frame for action button
        action_frame = ttk.Frame(top, padding="10")
        action_frame.pack(fill=tk.X)

        # --- Videojuego Selection ---
        ttk.Label(selection_frame, text="Videojuego:").grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        juegos_disponibles = self.manager.get_catalogo_videojuegos()
        juego_options = []
        self.juego_map = {} # Store on self to be accessible in submit_venta
        if juegos_disponibles:
            for j in juegos_disponibles:
                display_name = f"{j['nombre']} (${j['precio']:.2f})"
                juego_options.append(display_name)
                self.juego_map[display_name] = {'id': j['id'], 'nombre': j['nombre'], 'precio': j['precio']}
        
        self.selected_juego_str = tk.StringVar()
        juego_combo = ttk.Combobox(selection_frame, textvariable=self.selected_juego_str, values=juego_options, width=40, state="readonly")
        juego_combo.grid(row=0, column=1, padx=5, pady=5, sticky=tk.EW)
        if juego_options:
            juego_combo.current(0)

        # --- Cliente Selection ---
        ttk.Label(selection_frame, text="Cliente:").grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)
        clientes_disponibles = self.manager.get_clientes_for_selection()
        cliente_options = []
        self.cliente_map = {} # Store on self
        if clientes_disponibles:
            for c in clientes_disponibles:
                display_name = f"{c['nombre']} (ID: {c['id']})" # Email could be None
                cliente_options.append(display_name)
                self.cliente_map[display_name] = c['id']
        
        self.selected_cliente_str = tk.StringVar()
        cliente_combo = ttk.Combobox(selection_frame, textvariable=self.selected_cliente_str, values=cliente_options, width=40, state="readonly")
        cliente_combo.grid(row=1, column=1, padx=5, pady=5, sticky=tk.EW)
        if cliente_options:
            cliente_combo.current(0)
        
        selection_frame.grid_columnconfigure(1, weight=1) # Make comboboxes expand

        def submit_venta():
            juego_display_name = self.selected_juego_str.get()
            cliente_display_name = self.selected_cliente_str.get()

            if not juego_display_name or not cliente_display_name:
                messagebox.showerror("Error", "Debe seleccionar un videojuego y un cliente.", parent=top)
                return

            juego_seleccionado = self.juego_map.get(juego_display_name)
            id_cliente_seleccionado = self.cliente_map.get(cliente_display_name)

            if not juego_seleccionado or id_cliente_seleccionado is None: # id_cliente can be a valid ID (even 0 if DB allows)
                messagebox.showerror("Error", "Selección inválida. Verifique los datos.", parent=top)
                return
            
            confirm = messagebox.askyesno("Confirmar Venta", 
                                          f"Vender '{juego_seleccionado['nombre']}' por ${juego_seleccionado['precio']:.2f}\n"
                                          f"al cliente seleccionado (ID: {id_cliente_seleccionado})?", 
                                          parent=top)
            if not confirm:
                return

            success, message = self.manager.realizar_venta_gui(
                juego_seleccionado['id'], 
                juego_seleccionado['nombre'], 
                juego_seleccionado['precio'], 
                id_cliente_seleccionado
            )

            if success:
                messagebox.showinfo("Éxito", message, parent=top)
                top.destroy()
                self.show_catalogo_in_content_frame() # Refresh main catalog
            else:
                messagebox.showerror("Error en Venta", message, parent=top)

        ttk.Button(action_frame, text="Confirmar Venta", command=submit_venta).pack(pady=10)
        
        top.transient(self.root)
        top.grab_set()
        self.root.wait_window(top)


    def open_informes_window(self):
        if not self.manager.is_connected():
            messagebox.showerror("Error", "No hay conexión a la base de datos.")
            return

        top = tk.Toplevel(self.root)
        top.title("Informes de Ventas")
        top.geometry("700x500")

        informe_buttons_frame = ttk.Frame(top, padding="10")
        informe_buttons_frame.pack(fill=tk.X)

        informe_display_frame = ttk.Frame(top, padding="10")
        informe_display_frame.pack(expand=True, fill=tk.BOTH)
        
        # Use a specific container for report content to easily clear and update
        self.report_container = ttk.Frame(informe_display_frame) 
        self.report_container.pack(expand=True, fill=tk.BOTH)


        def show_all_sales_report():
            self.clear_report_container()
            sales_data = self.manager.get_all_sales_report()
            if sales_data is None: # Error fetching
                ttk.Label(self.report_container, text="Error al obtener el informe de todas las ventas.", foreground="red").pack()
                return
            if not sales_data: # No sales
                ttk.Label(self.report_container, text="No hay ventas registradas.").pack()
                return

            cols = ('ID Venta', 'Cliente', 'Videojuego Vendido', 'Precio Venta', 'Fecha')
            tree = ttk.Treeview(self.report_container, columns=cols, show='headings')
            for col in cols:
                tree.heading(col, text=col)
                tree.column(col, width=120, anchor=tk.W)
            tree.column('Precio Venta', anchor=tk.E, width=100)
            tree.column('Fecha', width=150)


            for venta in sales_data:
                tree.insert("", tk.END, values=(
                    venta['id'],
                    venta.get('nombre_cliente', 'N/A'), # Handle if client was deleted (ON DELETE SET NULL)
                    venta['nombre_videojuego_vendido'],
                    f"${venta['precio_venta']:.2f}",
                    venta['fecha_venta'].strftime('%Y-%m-%d %H:%M:%S') if venta.get('fecha_venta') else 'N/A'
                ))
            
            vsb = ttk.Scrollbar(self.report_container, orient="vertical", command=tree.yview)
            tree.configure(yscrollcommand=vsb.set)
            tree.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)
            vsb.pack(side=tk.RIGHT, fill=tk.Y)

        def show_total_revenue():
            self.clear_report_container()
            total, message = self.manager.get_total_revenue_report()
            if total is not None:
                ttk.Label(self.report_container, text=f"Ingresos Totales: ${total:.2f}", font=("Arial", 14)).pack(pady=20)
            else: # Error
                ttk.Label(self.report_container, text=message, foreground="red", font=("Arial", 14)).pack(pady=20)


        def show_sales_by_client():
            self.clear_report_container()
            data, message = self.manager.get_sales_by_client_report()

            if data is None: # Error
                ttk.Label(self.report_container, text=message, foreground="red").pack()
                return
            if not data: # Empty list (no sales grouped by client)
                ttk.Label(self.report_container, text="No hay ventas agrupadas por cliente para mostrar.").pack()
                return

            cols = ('Cliente', 'Nº Ventas', 'Total Gastado')
            tree = ttk.Treeview(self.report_container, columns=cols, show='headings')
            for col in cols:
                tree.heading(col, text=col)
                tree.column(col, width=150, anchor=tk.W)
            tree.column('Total Gastado', anchor=tk.E, width=120)
            tree.column('Nº Ventas', anchor=tk.CENTER, width=80)


            for row in data:
                tree.insert("", tk.END, values=(
                    row['nombre_cliente'],
                    row['numero_ventas'],
                    f"${row['total_gastado']:.2f}"
                ))
            
            vsb = ttk.Scrollbar(self.report_container, orient="vertical", command=tree.yview)
            tree.configure(yscrollcommand=vsb.set)
            tree.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)
            vsb.pack(side=tk.RIGHT, fill=tk.Y)


        ttk.Button(informe_buttons_frame, text="Todas las Ventas", command=show_all_sales_report).pack(side=tk.LEFT, padx=5)
        ttk.Button(informe_buttons_frame, text="Ingresos Totales", command=show_total_revenue).pack(side=tk.LEFT, padx=5)
        ttk.Button(informe_buttons_frame, text="Ventas por Cliente", command=show_sales_by_client).pack(side=tk.LEFT, padx=5)
        
        top.transient(self.root)
        # top.grab_set() # Decided against making it modal for now, user might want to see main window
        # self.root.wait_window(top) # If not grab_set, this might not be what's wanted.
                                   # If it should be modal, uncomment grab_set and wait_window.
                                   # For non-modal, just let it be a normal Toplevel.

    def clear_report_container(self):
        for widget in self.report_container.winfo_children():
            widget.destroy()


# --- Main execution ---
if __name__ == "__main__":
    root = tk.Tk()
    app = GameStoreApp(root)
    
    def on_closing():
        if messagebox.askokcancel("Salir", "¿Seguro que quieres salir?"):
            if app.manager: # Ensure manager was initialized
                app.manager.cerrar_conexion()
            root.destroy()

    root.protocol("WM_DELETE_WINDOW", on_closing) # Handle window close button
    root.mainloop()


GameStoreManager: Conectado a la base de datos.
GameStoreManager: Tabla 'videojuegos' verificada/creada.
GameStoreManager: Tabla 'clientes' verificada/creada.
GameStoreManager: Tabla 'ventas' verificada/creada.
